In [ ]:
import pandas as pd
import geoviews as gv
import geoviews.feature as gf
from bokeh.sampledata.airport_routes import airports, routes

gv.extension('matplotlib')

gv.output(fig='svg')

## Define data

In [ ]:
# Count the number of connections from each airport
counts = routes.groupby('SourceID')[['Stops']].count().reset_index().rename(columns={'Stops': 'Connections'})
airports_df = pd.merge(airports, counts, left_on='AirportID', right_on='SourceID', how='left')

# Select only US mainland airports & convert from Mercator to Latitudes/Longitudes
airport_points = gv.Points(airports_df, ['Longitude', 'Latitude'])[-170:-50, 0: 50]

# Declare nodes, graph and tiles
nodes = gv.Nodes(airport_points, ['Longitude', 'Latitude', 'AirportID'],
                 ['Name', 'City', 'Connections'])
graph = gv.Graph((routes, nodes), ['SourceID', 'DestinationID'], ['Source', 'Destination'])
tiles = gv.tile_sources.Wikipedia

# Select 50 busiest airports
busiest = list(routes.groupby('SourceID').count().sort_values('Stops').iloc[-50:].index.values)
busiest_airports = graph.select(AirportID=busiest, selection_mode='nodes')

## Plot

In [ ]:
gf.ocean * gf.land * gf.coastline * gf.borders * busiest_airports.opts(
    node_size=8, edge_linewidth=1, edge_alpha=0.05, fig_size=300, padding=0.1)